In [1]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from cStringIO import StringIO
import nltk, string
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
def convertPDFToText(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = file(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()
    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    fp.close()
    device.close()
    string = retstr.getvalue()
    retstr.close()
    return string

In [3]:
def convertDocxToText(path):
    document = Document(path)
    return "\n".join([para.text for para in document.paragraphs])

In [17]:
text1 = convertPDFToText('syllabus/host/diploma_host/DBS5028.pdf')
text2 = convertPDFToText('syllabus/host/foundation_host/PBU0045.pdf')
# file = open('test_syllabus/Diploma.txt',mode='r')
# text2 = file.read()
# file.close()

In [14]:
import re
lines_ini = re.split('\n',text1)
lines_ini

['1 .',
 '2 .',
 '3 .',
 '',
 'Name of Course',
 'Course Code',
 'Type of Course',
 '(e.g. : Core, major, elective etc.)',
 '',
 '4 .',
 '',
 'Synopsis',
 '',
 'COURSE INFORMATION',
 '',
 'E-Commerce',
 'DBS5028 ',
 'Core for Diploma in Business Administration',
 'Elective for Diploma in Information Technology, Diploma in Business ',
 'Information Systems, Diploma in Accounting, Diploma in Electronics ',
 'Engineering',
 'This course provides an overview of e-commerce that covers Web ',
 'marketing and advertising, mobile commerce, social commerce, e-',
 'commerce security and payment systems. Students learn the various types ',
 'of e-commerce business models, revenue models and the process of ',
 'starting a new online business.',
 '',
 '5 .',
 '',
 'Version',
 '(State the date of theSenate\xe2\x80\x99s approval - previous and the current approval date)',
 '',
 '6 .',
 '',
 'Name(s) of Academic Staff',
 '',
 'Semester and Year Offered',
 'Credit Value',
 'Pre-Requisite',
 'Objective 

In [18]:
lines_ini = re.split('\n',text2)
lines_ini

['1. ',
 '2. ',
 '3. ',
 '4. ',
 '5. ',
 '',
 '6. ',
 '',
 '7. ',
 '',
 '8. ',
 '',
 '9. ',
 '',
 '10. ',
 '11. ',
 '12. ',
 '',
 '13. ',
 '',
 '14. ',
 '15. ',
 '',
 '16. ',
 '',
 ' Name of Subject : Introduction To Business Management (Management Student Only) ',
 ' Subject Code: PBU0045 ',
 ' Status of Subject: Core ',
 ' Stage: Foundation  ',
 ' Version: Date of Previous Version:- October 2011 ',
 '               Date of Current Version \xe2\x80\x93 May 2012 ',
 ' Name (s) of academic staff :  ',
 'Arifah Hamzah,  Noridayu Abdullah Sani, Norzarina Md.Yatim, Kerk Peck Hoon,Kausar Yaakup, Lau ',
 'Siok Hwa,Zalina Jaafar, Nurul Afidah, Sarinah Suliman, Teo Kim Mui, Rekha Thangatorai   ',
 ' Rationale for the inclusion of the subject in the programme : ',
 'To provide students with the understanding of principles of business and management and the ability ',
 'to identify, formulate and solve business and management related problems. ',
 ' Semester and Year offered : Trimester 2 ',
 ''

In [14]:
stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

def token_stemming(tokens):
    return [stemmer.stem(item) for item in tokens]

# remove noise like punctuations and convert everything to lower case
def noice_removal(text):
    return token_stemming(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

vectorizer = TfidfVectorizer(tokenizer=noice_removal, stop_words='english')

def similarity_cosine(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]


print(similarity_cosine(text1, text2))

0.195030959442
